<a href="https://colab.research.google.com/github/Mariyyah-Alrasheed/6_Week/blob/main/Another_copy_of_YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install opencv-python cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=4390fe19d5688f3c0548425ecb85e646c821f52dc411cb041cd6ddef1f52eac9
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 24.4 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO, solutions
import cv2
from PIL import Image

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [6]:
!yolo track task=detect model=/content/YOLO_Model.pt

WARNING ⚠️ 'source' argument is missing. Using default 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics YOLOv8.2.85 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 cars, 104.2ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 1 car, 50.5ms
Speed: 6.8ms preprocess, 77.4ms inference, 337.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/track
💡 Learn more at https://docs.ultralytics.

In [7]:
model = YOLO("/content/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [8]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [9]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [10]:
output_path = '/content/Output_YOLO_Video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [11]:
counter= solutions.ObjectCounter(
    names=model.names,
    reg_pts=[(150,height//2), (width//2 - 120, height//2)],
    count_reg_color=(0, 0, 255),
    count_txt_color=(0, 0, 0),
    count_bg_color=(255, 255, 255),
    line_thickness=2,
    view_img=True,
    view_in_counts=True,
    view_out_counts=True,
    draw_tracks = False,
    region_thickness=5,
    line_dist_thresh=50,
    cls_txtdisplay_gap=50,
)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


In [12]:

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print('Video frame is empty or video processing has been successfully completed.')
        break

    # Perform object tracking
    tracks = model.track(frame, persist=True, show=False)

    frame = tracks[0].plot()
    frame = counter.start_counting(frame, tracks)

    output.write(frame)

cap.release()
output.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 67.3ms
Speed: 3.3ms preprocess, 67.3ms inference, 511.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 53.6ms
Speed: 2.9ms preprocess, 53.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 54.6ms
Speed: 3.0ms preprocess, 54.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.1ms preprocess, 53.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 53.5ms
Speed: 2.4ms preprocess, 53.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.1ms preprocess, 53.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.4ms preprocess, 53.5ms infere

In [ ]:


# while cap.isOpened():
#   ret, frame = cap.read()
#   if not ret:
#     break

#   frame = cv2.resize(frame, (1920,1080))
#   results = model.track(video_path)
#   current_detections = np.empty([0,5])

#   for result in results:
#     boxes = result.boxes.xyxy.cup().numpy()
#     confs = result.boxes.conf.cpu().numpy()
#     class_ids = result.boxes.cls.cpu().numpy()

#   for box, conf, class_id in zip(boxes, confs, class_ids):
#     x1, y1, x2, y2 = map(int, box)
#     label = f'{model.names[int(class_id)]} : {conf:.2f}'

#     cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
#     cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
#     out.write(frame)
# cap.release()
# out.release()

# print(f"Video saved to {output_path}")


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

# Try 2

In [15]:
output_path_2 = '/content/Try_2_Output_YOLO_Video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_2 = cv2.VideoWriter(output_path_2, fourcc, fps, (width, height))

In [ ]:
# make a line pionts for evry side

In [16]:
# create 4 counters

heigh_fourth = height//4
width_fourth = width//4

counter_bottom = solutions.ObjectCounter(
    names=model.names,
    reg_pts=[(150,heigh_fourth), (width//2 - 120, heigh_fourth)],
    count_reg_color=(0, 0, 255),
    count_txt_color=(0, 0, 0),
    count_bg_color=(255, 255, 255),
    line_thickness=2,
    view_img=True,
    view_in_counts=True,
    view_out_counts=True,
    draw_tracks = False,
    region_thickness=5,
    line_dist_thresh=50,
    cls_txtdisplay_gap=50,
)

counter_right = solutions.ObjectCounter(
    names=model.names,
    reg_pts=[(width - width_fourth,height//2), (width_fourth, 0)],
    count_reg_color=(0, 0, 255),
    count_txt_color=(0, 0, 0),
    count_bg_color=(255, 255, 255),
    line_thickness=2,
    view_img=True,
    view_in_counts=True,
    view_out_counts=True,
    draw_tracks = False,
    region_thickness=5,
    line_dist_thresh=50,
    cls_txtdisplay_gap=50,
)

counter_top = solutions.ObjectCounter(
    names=model.names,
    reg_pts=[(width, height - heigh_fourth), (width//2, height - heigh_fourth)],
    count_reg_color=(0, 0, 255),
    count_txt_color=(0, 0, 0),
    count_bg_color=(255, 255, 255),
    line_thickness=2,
    view_img=True,
    view_in_counts=True,
    view_out_counts=True,
    draw_tracks = False,
    region_thickness=5,
    line_dist_thresh=50,
    cls_txtdisplay_gap=50,
)


counter_left = solutions.ObjectCounter(
    names=model.names,
    reg_pts=[(width_fourth,height), (width_fourth, height//2)],
    count_reg_color=(0, 0, 255),
    count_txt_color=(0, 0, 0),
    count_bg_color=(255, 255, 255),
    line_thickness=2,
    view_img=True,
    view_in_counts=True,
    view_out_counts=True,
    draw_tracks = False,
    region_thickness=5,
    line_dist_thresh=50,
    cls_txtdisplay_gap=50,
)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


In [18]:

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print('Video frame is empty or video processing has been successfully completed.')
        break

    # Perform object tracking
    tracks = model.track(frame, persist=True, show=False)

    frame = tracks[0].plot()
    frame = counter_bottom.start_counting(frame, tracks)
    frame = counter_right.start_counting(frame, tracks)
    frame = counter_top.start_counting(frame, tracks)
    frame = counter_left.start_counting(frame, tracks)

    output_2.write(frame)

cap.release()
output_2.release()
cv2.destroyAllWindows()

# Try 3

In [ ]:
# Here ii try to do a solution i found it on google by using cvzone
# this is the link :
# https://github.com/Tech-Watt/Vehicle-Counting-in-Lanes/blob/main/Vehicle%20Counting%20in%20Lanes.py

In [ ]:
output_path = '/content/Output_test2_YOLO_Video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
import cv2
import cvzone
import math
import numpy as np
from ultralytics import YOLO
from sort import *

In [ ]:
video_path = '/content/Output_YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)


In [ ]:
# # here I will modify the pionts to mine

# road_zoneA = np.array([[308, 789], [711, 807], [694, 492], [415, 492], [309, 790]], np.int32)
# road_zoneB = np.array([[727, 797], [1123, 812], [1001, 516], [741, 525], [730, 795]], np.int32)
# road_zoneC = np.array([[1116, 701], [1533, 581], [1236, 367], [1009, 442], [1122, 698]], np.int32)


road_zone_bottom = np.array([[150, heigh_fourth], [width//2 - 120, heigh_fourth], [width//2, 0]], np.int32)
road_zone_right = np.array([[width - width_fourth, height//2], [width_fourth, 0], [width, height//2], [width, 0]], np.int32)
road_zone_top = np.array([[width, height - heigh_fourth], [width//2, height - heigh_fourth], [width, height], [width//2, height]], np.int32)
road_zone_left = np.array([[width_fourth, height], [width_fourth,height//2], [0, height//2], [0, height]], np.int32)


In [ ]:

zonebottom = np.array([road_zone_bottom[0],road_zone_bottom[1]]).reshape(-1)
zoneright = np.array([road_zone_right[0],road_zone_right[1]]).reshape(-1)
zonetop = np.array([road_zone_top[0],road_zone_top[1]]).reshape(-1)
zoneleft = np.array([road_zone_left[0],road_zone_left[1]]).reshape(-1)

tracker = Sort()
zoneAcounter = []
zoneBcounter = []
zoneCcounter = []

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1920,1080))
    results = model(frame)
    current_detections = np.empty([0,5])

    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)
            cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            if class_detect == 'car' or class_detect == 'truck' or class_detect == 'bus'\
                    and conf > 60:
                detections = np.array([x1,y1,x2,y2,conf])
                current_detections = np.vstack([current_detections,detections])

    cv2.polylines(frame,[road_zoneA], isClosed=False, color=(0, 0, 255), thickness=8)
    cv2.polylines(frame, [road_zoneB], isClosed=False, color=(0, 255, 255), thickness=8)
    cv2.polylines(frame, [road_zoneC], isClosed=False, color=(255,0, 0), thickness=8)

    track_results = tracker.update(current_detections)
    for result in track_results:
        x1,y1,x2,y2,id = result
        x1,y1,x2,y2,id = int(x1),int(y1),int(x2),int(y2),int(id)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2 -40


        if zoneA_Line[0] < cx < zoneA_Line[2] and zoneA_Line[1] - 20 < cy < zoneA_Line[1] + 20:
            if zoneAcounter.count(id) == 0:
                zoneAcounter.append(id)

        if zoneB_Line[0] < cx < zoneB_Line[2] and zoneB_Line[1] - 20 < cy < zoneB_Line[1] + 20:
            if zoneBcounter.count(id) == 0:
                zoneBcounter.append(id)

        if zoneC_Line[0] < cx < zoneC_Line[2] and zoneC_Line[1] - 20 < cy < zoneC_Line[1] + 20:
            if zoneCcounter.count(id) == 0:
                zoneCcounter.append(id)

        cv2.circle(frame,(970,90),15,(0,0,255),-1)
        cv2.circle(frame,(970,130),15,(0,255,255),-1)
        cv2.circle(frame,(970,170),15,(255,0,0),-1)
        cvzone.putTextRect(frame, f'LANE A Vehicles ={len(zoneAcounter)}', [1000, 99], thickness=4, scale=2.3, border=2)
        cvzone.putTextRect(frame, f'LANE B Vehicles ={len(zoneBcounter)}', [1000, 140], thickness=4, scale=2.3, border=2)
        cvzone.putTextRect(frame, f'LANE C Vehicles ={len(zoneCcounter)}', [1000, 180], thickness=4, scale=2.3, border=2)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video